## Final Project

+ You can form teams of up to 4 people

+ Consider the following 5 stocks:
    1. Apple(AAPL)
    2. AT&T (T)
    3. NETFLIX (NFLX)
    4. General Electric (GE)
    5. Boeing (BA)

+ Using Yahoo finance, for each of these stocks, consider statements under the “Conversation” tab that are at most 5 days old. Create a lexicon (dictionary) of words that show positive sentiments and negative sentiments.

+ Consider the vector (stock, feature, time, sentiment) where:
    * Stock = AAPL, T, NFLX, GE, BA
    * Feature = can be important items such as sales, dividend, grounding
    * Time = in days where 0 is today, 1, yesterday, 4 is five days ago
    * Sentiment = Positive (1), Negative (-1), Neutral (0).

+ Determine this vector for all the statements of the “Conversations” that are about that particular stock and put them in an EXCEL sheet.

+ Do a pie chart of sentiment for each of the 5 days and based on that determine whether the sentiment about the stock is positive, negative, or neutral.

+ Plot the sentiment for the past five days vs Open, Close, High, Low, and Volume of the stock.

+ Does there seem to be a correlation between sentiment and the price, volume and volatility.

+ You can also perform other analysis that is of interest to the group
    * I will ask the team to present its analysis about one of the stocks.
    * The team will submit its work that should include the name of team members.